# Ain't No Mountain High Enough

The cipher is made of 5 keys, `K1` to `K5`, which are 5×5 matrixes.
It can be summed up by the equation: `enc(M) = K5 * K4 * K3 * K2 * K1 * M`.
Additionally, we can replace any key `Ki` by the flag `FL` and request encryption.

In [1]:
N = 5
P = 251
K = GF(P)

In [2]:
def pad(msg, n):
    # Bytestrings only pls
    if type(msg) == str:
        msg = msg.encode()
    # Apply random padding
    while (len(msg) % (n*n) != 0):
        msg += os.urandom(1)
    # Matrixfy and return
    return [matrix([list(i[j:j+n]) for j in range(0,n*n,n)]).change_ring(K)
            for i in [msg[k:k+n*n] for k in range(0,len(msg),n*n)]]

In [3]:
def encMC(msg, key):
    # Convert msg to matrix
    if type(msg) in { str, bytes, list }:
        msg = pad(msg, N)

    # For all nxn msg matrices
    ct = []
    for mi in msg:
        # Hill-cipher encrypt with every key matrix
        for i in range(N):
            mi = key[i] * mi
        # Add result to ciphertext
        ct += [mi]

    # Return ciphertext in hex
    return ct

A very important property of this cipher is that if `E = enc(M)`, then `enc(M * X) = E * X`. This means that if we set `M = Id` we can just ask the server once for `E = enc(Id)` and do everything locally afterward.

In [4]:
M = MatrixSpace(K, N)
m = M.identity_matrix()
m

[1 0 0 0 0]
[0 1 0 0 0]
[0 0 1 0 0]
[0 0 0 1 0]
[0 0 0 0 1]

In [5]:
msg = ''.join(map(chr, m.list()))
msg

'\x01\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x01'

The data was gathered using [`get_data.py`](./get_data.py).

In [6]:
data = [
    b'|   [134, 135, 128,  42, 181]\n',
    b'|   [ 82,   6,  60, 212, 103]\n',
    b'|   [104, 130,  29,  12,  77]\n',
    b'|   [167, 118, 190,  17, 162]\n',
    b'|   [198,  73,  95, 219, 178]\n',
]

craw = pad(sum([eval(x[1:].strip().decode()) for x in data], []), N)[0]
# craw = K5 * K4 * K3 * K2 * K1
craw

[134 135 128  42 181]
[ 82   6  60 212 103]
[104 130  29  12  77]
[167 118 190  17 162]
[198  73  95 219 178]

In [7]:
data = [
    b'|   [176, 106, 194,  92,  83]\n',
    b'|   [142, 143, 149,   2, 233]\n',
    b'|   [150,  52,  87, 154, 177]\n',
    b'|   [ 77, 219,  13, 235, 112]\n',
    b'|   [ 73,  83,   8, 245, 166]\n',
    
    b'|   [152, 233,  74, 169, 143]\n',
    b'|   [  6,  66,  71,  46, 169]\n',
    b'|   [117,  98,  54, 241, 166]\n',
    b'|   [209, 149,  25,  62,  45]\n',
    b'|   [236,  44, 101, 137, 244]\n',
    
    b'|   [100, 214,  47,  28, 217]\n',
    b'|   [ 72, 220, 189, 143,  92]\n',
    b'|   [200,  73, 165,  12, 138]\n',
    b'|   [ 99, 139, 222,  89, 131]\n',
    b'|   [106, 214, 149,  94, 186]\n',
    
    b'|   [ 97,  77, 208,  34, 239]\n',
    b'|   [191, 236,  13,   8,  53]\n',
    b'|   [ 69, 229, 141, 248, 132]\n',
    b'|   [233, 152, 185, 119, 192]\n',
    b'|   [132,  77,  69,  76,  78]\n',
    
    b'|   [ 30,  19,  95, 220, 219]\n',
    b'|   [153,  17, 137, 245, 215]\n',
    b'|   [ 44,  16,  87,   6,  86]\n',
    b'|   [ 66, 111, 173, 207,  64]\n',
    b'|   [139,   7,  65, 249, 233]\n',
]

cflag = pad(sum([eval(x[1:].strip().decode()) for x in data], []), N)
# cflag = [
#     FL * K4 * K3 * K2 * K1,
#     K5 * FL * K3 * K2 * K1,
#     K5 * K4 * FL * K2 * K1,
#     K5 * K4 * K3 * FL * K1,
#     K5 * K4 * K3 * K2 * FL,
# ]

In our quest to retrieve `FL`, we might try to multiply the content of `cflag` by `craw^-1` in order to remove the noise at the right.

We would get the following entries:
```
(1) FL * K5^-1
(2) K5 * FL * K4^-1 * K5^-1
(3) K5 * K4 * FL * K3^-1 * K4^-1 * K5^-1
(4) K5 * K4 * K3 * FL * K2^-1 * K3^-1 * K4^-1 * K5^-1
(5): K5 * K4 * K3 * K2 * FL * craw^-1
```

It is interesting to see that the trailing noise of `(i)` is the inverse of the leading noise of `(i+1)`:
```
  (1) * (2) * (3) * (4) * (5)
= (FL * K5^-1) * (K5 * FL * K4^-1 * K5^-1) * (3) * (4) * (5)
= FL^2 * K4^-1 * K5^-1 * (K5 * K4 * FL * K3^-1 * K4^-1 * K5^-1) * (4) * (5)
= ...
= FL^5 * craw^-1
```

If we multiply this quantity by `craw`, we will thus know `FL^5`.

In [8]:
FL5 = prod(cf * craw^-1 for cf in cflag) * craw
FL5

[ 62 170 162 116  44]
[ 95 160 148 199 165]
[174 159 192   9 105]
[ 38  24 216 115 108]
[198  13  17 201  51]

Knowing `X^5`, how can we know `X`? When `X = P*D*P^-1` with `D` a diagonal matrix, `X^(1/5) = P*D^(1/5)*P^-1`. Hence we can know one 5th root of `X` by searching fifth roots of the diagonal of `D`.

In [15]:
D, P = FL5.diagonalization()
D

[235   0   0   0   0]
[  0 200   0   0   0]
[  0   0  91   0   0]
[  0   0   0  50   0]
[  0   0   0   0   4]

In [10]:
all_diag = [d.nth_root(5, all=True) for d in D.diagonal()]
all_diag

[[129, 139, 70, 19, 145],
 [143, 193, 99, 95, 223],
 [101, 31, 12, 118, 240],
 [228, 234, 42, 162, 87],
 [105, 154, 92, 68, 83]]

In [18]:
from itertools import product

for diag in product(*all_diag):
    FL = P * M.diagonal_matrix(diag) * P^-1
    assert FL^5 == FL5
    
    flag = ''.join(chr(x%251) for x in FL.list())
    
    

Öu¶`_¸¿«¤öR"½#*§1U
Ýnt¹éF8Ý ð¬ ñ·Ãí%
oÏ¾Y§éÏ1yëäµ5TS_C
8ä.bQG½pÚÓ=`w= WKo
;·?H7íÎëÍtï	âPÞ¤(Ú­Ñ
Kø&ãôOãoPF°£~ªõªîø*ð
eÑâR¶£U6ðÎúÕ¤ni.ÆOaCõÀ
ß$( 1íty¤^úã¾iò½(æ[VÞ
¨l?®¿.ÜÒsøøaÒ­ì"¯L§ çD
«?P¥{Ü½lYà×<¤¹³¯£m¦q
H÷6îpfØèZu¨W\÷*Ò¨uñ¦1
A°U$Üs9éÎ@Nd¸®VÜdìÚ
oùn+ò¿"GÍäèÚ\h>±^ø
b·¹O_},lK×
¥br#)K
e&â§ÁZÇÈÊ°éXóÉz¦­
í¶UD³<Vë}wæPNKÙ8~
.#®2¡?)cöP>_§HedØcNY
é¸ÜotùvlwJI§³H
O*n~O0 8.F75¾£
RøðdsÔê¢a©Û´lú=Fú

\°ùz;Ú`#Ý7¿§l5!Ê ÍXR
Wh)Ý3	ÃÅÃ{¨´Ï @Ø("
ð× ²s}ìhX	ñ¬uÒ1<:ëF@
¹$E¹YÁFéXdZ/J:Uøúrl
¼ò(+çjO]{×Å¥±ø¡¼ÔÎ
h¸3'H°£ÖTxrB4ËÜ4íAâ?
¤%6³vç:^`Å`[Å;gJh¯-é
ß5_S4½­Y3tD¿»§Å.©@-
Å,Líw>Íîc3ê	ËéênÑY8
Èú]Óôç[¿Áå­.r]]q·7UòñW»
Ø@Ds¶Ipf)hnW3,ÙòNCy
88¹
¨a¤LCwNGªQ5yøÐwx
qgF«îçM^@}l>)ßÔµr&§
Á¬øv7'RÓL%²ß[
=n$g(OÕc73%od~º¨´:
áõÁ°jî¬¢rÆ1²1+YD
Îós¯mÁ½Xv;+ào_ä{,èÅ
×§ogqòëiÃ¼êu?ã
ïú3{IçìÅT&É`ásàA:Ð7
òÍDra¡¦ßÃM¾¿LzI½V

öÐ{ðaÜu&@jf4ccÈo¨
¿GNOÊZÀ¢Yé¾}TÈÔ
Âë|ïÁòÉ!&89E0îî\/;
OG4ù!¼b¥?ÀÓ¿^F "
 Çñ7Ì2¢H~ÞÜ/<z7ºnÎíâ
ãCI@Òñw ÅX¢×Vhá
¬`¤Îòaß\q´ÛÍÙ ¸-w<1
¯^q´k±
.ÏËöðÞ1+Tg$°ø
¹ë²b/eé¸cÕ¢{ÀÝ<÷Û
õ~IyÕÖÃH>k!E]4¯ôªHÇ«
R=íLÃud7ÔM.ø"¸ÔV¤[åÉ
	ÚV±òRs¯ç¥ã"5zÚiìõ
XÀ<»ÏÏ3GðjÐáâìrx\
°Âìù÷hX|Ñ$ç+Wñí¤!
ìlß°ÚÊy>bhÞïï{á5¾N`êìê
I+ú)O±aL Ød¦ «2Z
s·Øð¨{FæÇS.9ÒDÏî;9
F°s5¯QÅéµGñ}y»«;ö¢
%_Äó-lvpÂùèÀÂ)}¶õº
aïðM	ROÃàçUGö]")@å
¹®K5UÚb¡H1Ð>ZËòXº¹#¨
ö°Ù*q½8P-Ës¿-Ýö@|Þzãä4Ô
ÉÁ¿É|ÜÙ,òéW-}îãJkj;
ÚÒmaY³G>¦v¥ß¬ÒMÐ_?H¹£úö
?ÆOG¶O\~7Êì.G/=Ósè,éÊÆ
sùoVÓ%Û«w öÕ³5Ð&èä
<F,$@~É\?]ÄÞÚYôEæ
?
êú'ãËÜî0ÚeÀ`Mn{w
ÇEÆo7Q;1§¹{ðË?ß»pYnS
­$]
b!L'9>L¤«Ç`>#
`lÈ§V"Ã8pfyzÅ5ùÂí©GÁs\A
)´ß:ä^0^T^,$úí;Í	m
,ð Ê¶ê:¨Íu«N)Ìté9êÏ
6?o¥¡"AâÜùg¹Y°	õM2ÎH
r§È%óÂß@å×°TÝfÕç
ÊfqÝÙÀÍÉ39ðÃØ6ÞÃè6

®pl:'!íjßçËÂ~b6
VRYôËNkNé²2nSÓJÄ
¶ Î*
9ÙãÙ´(ßLO
êÉ¿kâ¹

ÅäösvÎóÔ¿¡2ÂÉéù"¯
Pë#Òävg¦ä\§Kòà Ó<\Dí
¨ªÇ!3%|8« ´#É`ô_5ÓVW
qòÞ¯ÁRÕ&S¸îã@BYè<É
tÅï§ªL~pãô¯	â3ÂðêÀn0
L.õ´gåÝÙÜóÉôTøW1µ
Ð(f:ÈÅMË¶1úOî÷(ÒÍÅÑ
-âf²¯+8×¼¬+å/9¿ä£
9'ÆD×tSõzKÏ
ô7øÚK­ÔQð^'uº ­6
	C¨ÂD<ÕÄ¾Íëq±b¨ Ü*U
E«w«U"»ëGÈ«å@ÀÜ)'õ%
j ÐÔKS+q0±pn">À:C
f²7cg»_ø{~ óÝFbMËDo
iHIMßz-©GêzIÊC ­ÉÐQ¦Ñ
¾ïæÉEµßó2·ðJcsi¾O#
úöMÔÌÅÌÕL¡]ÍÚòïÐÚa
Wµñ#ÎlÑqêÅî5"V3úTãø
±a¨>ÊØªh$y¥|#xLPy)«
#ÐGøs'QéçN£/ßTÓú
«Mô[5JõPCU]Ýï¾~é
çi¦âI®0Î_¤§OÜ;(²1GN¹
D(O18ÁÏÇÈÐesN+Zl×
pf¿&t.Ým½¬g7rØ©ëë
Cw¥Ìè· Ã¶©âsqúj
öñ/Þ8?3ëgµàÄ´(¡±ÕúvX²
VcOÌ;DÑM8âòKåzm¼(
®"ógÖË%¡ôhÙ{Ó¨GgÏF 
wjõ©8sl@uãWa¦!ÌkÒ,erÌ
z= Ûoò]ÔÛô¬-Ï8ÒÚ¯æÔ3
Ïo_=Ì5ÑÏHæzÖÈ-O-@NU
}ÈM+F·µå	Ñ2';µ%
h<qu:]	Þ	5ºQ«_àQ<C
1vÅuôòÃ£©
ÔÖ¨¼	Z/ho
4WéÎCA7Ó4$][(pÝ°ÊÑ
D««0À¦¿ó-×r°F?^ñGð
Ù3lÑ¥ÌõÉ@ätsAd×À
Ø¿ããÎ*§ÛôÅé&²ðmÈÐïØ^ê5Þ
¡vvÉâª¡ßô#a
¤Úª\\gQ©1ä

­yca]+	ù¨ë¬Qx2%D
Úg 6räC\±U_Éàðxð
7@¯±Ó»ïH8øÅÝ>èOã"Bâ´ê2
 ÆDf÷\¡&ÝdD-×Òkfu¯!?^
[×*LTJpÊQÀÃW"Ír}7¢¡À
`äSt-ÞbY¡L%;"¤
ÇïduNç&yù6°ø|n*Y¶Ûêdl
¿ß[ÈMú¬áAò~¶w
èö$R+ÃL´¶;ß>ÐÄ"ÇÚ<8O®¶
ëÉ58 ]S²ú¤zF£@Ò
õ¯½ã]
9àè· ½L*ZD^ne
«Ñ0ÆÁ¾ÝFnx£Ìd>5
ÂëÕ?Æw\SnºxêÆ
È®fVJQìµ8ù 
ZÃÙn¾î¦RsÐßÙº¾{êá
¯-íBæêMDÂ9¿ZJÝ>ã
Û0é½3ràÚ¹´õ~Þú.4Ó
HÂ/*z{îTÛÃy~=NÛszôARñ
ÅãLBpözs²hh;¹Ò~"
ÝWî"¢ðºjÖòÜQêJkCAXà
$#>>°·ï=+³{sìÅU9]£
`,¨Õ#WÑÒjmvc¶%jÃ-s
¸J@vè"H~)røÀ»S2ú·½K
W	{^°×`Ý_dªBµ*<¦Â(w½
ehêa¶oaªQ»ãÔl¬³£Ê
©Ù$
Ùnª :ÝmGZsç%$_XâBß
îS-3Æ»×C^;5Æ-¯
rÄâCÆÄ¢|,4çÀ,ßâVðÌÀ@0Í
;Ý-Wu3"jÎN¯g¼Ñ\ù
W¾`=[×íÆÚ´/ÍÙE²R(
ÆÜmol*ð\MlÛàÓ8SóY±<
IÆ]oøãã5 7<m^lóf
oÒ§¶×77.¶ %Ññ²iÉ`²*
(Pe:K#5%%¨Y! íP%HËV
+#K £ÝÙoo	9>¤£®ôYX¨É2¸
õ4Î4£Ê&ªDÅÜ)Ä
qCo¾à°ðyÈ!Ö¢A.»¢[Ð
/­ÀmØéâô
öW,yî
J/½é-ÆÆíÎ[sô#7DÂ¯a½¥
@£Fço<BÝÚ#n¹ò. äC
,Ä¬X6(<F}î1

IÍDÑ·Ty³Ð£´Ûe5òÖÁ¶Å
[dJát¾'ï£cµ®YC³Wâñ
÷Í®<aÀª»#ØIX
5Þ²RNëµà­Ö/L}Ce*×õ×Ý
qKÞê U!ð¹ô£wC^·¼J@§­
É
9êðÚÒêD²n%o<&»XDSÅË
^{¿Qàß=	äñ÷
%¯­cÔ'|i¥ÓK¤!jX^
¥fM%æ{úÇÇ'&1Îb£ KÐ}
áÎï;éépà åå~}+_¬ÆH M
¤¾k]§b¿4Sgfëãó(ß
Õ¯ëÅ»­"~íxVUsNL Í:ê

¨ÀùÑ"Îd÷líN÷¾³¨U»Qù
_±l	ö±,úÞèm?¨n£ôµ¹
lÒéêÏYÏVØGKsÜ?
óØnÓWª*¨=>ÈÂ l?ËÐ>ªR£§
¼%fåæ+,­ab[.%öèôbkÐãÏÓ
¿óLËCQªuv!½áXp}`ÉsXi6:
L$Å©ßÓÔ¿øÛóÐ÷É³Ù1Ck)
#ÍÖ\å~¥Ñ3ÞOÊV§çÑ±±ôá
àKÇáv»ÍôÊÊÇ8Ú Iom«Ä
©Þtª²»â¯;1¶'
Im.pZC0
¬fïZA½
1#°àQb÷Ôú6óÛ¥
¶nßgvj>¸´¾ëØèôàÚ
òÇÍUyf{$rÜG4lë!Ïí+Îª
0YðD~15ç>ìÈ
ª2Uªa¯ooÜ ¢Uñ¬Ïô
`­KS«ùÞpîI6,¢@¼ù4U[
p8çÁÕ%l$ôà±BØ94­ªô}
¬ E¯Øó}
âwmÔ ðÄM
	_éLùx±SY.áØ/ñËÌ%ËnÊpâk
Í§Ú´¬GÆwàºTPî1Ú
ÐzÀrØUf:ÓäÒÇùZ9buù
à»ø`4ní÷ôäöÑ©eÁH»v í
ÚÊÏTï¼»^àI|`äF½è
yâúl~Þ.È½ é¥â9¦Ù÷ÞYÛ
B/+úMæ<­\ØÊ¸£ê7
E'à¥¥àfV!¸¾S56iÀ+pn
Î®.pBrî*p»®>sç¡í^y©ÒY
Ös,CSXÔÃÙ
mF&+çï¢)
32Ðæf2)§(÷1
î~2ÊÓïÂá

^=ØÑç$[i\aM qp7Ó9ÌQL
FÆÆ¿êi5AìB¯¤wkqNGkx§!
?'@;«`0õ¢¤È¡*?:
gÍV£Æd~. ·|O³%ëÈìÐf»kf
Èx¦y§4XØéAÍÈ
LÓ7ÕQXÏ4÷õh%"çêsv
=´1y%]æ7>l'RSÞïÂVÀ]¹F[
sÕÃo±¼e~<Ç´KW³"(WÌdy
^»ìVít{J/à+¶<v ×Fäb¥
a<ãJB¾¾Å¹dUàøSC­ìãò
qÏä×¥§We&Hz´%ïa©*Zs³Äo+
­<BÅª*v.SCK¸£lÏ&?ö

Àë1¯ "]}Luo4¡h
ÎC¢pP¥Ik1Ñ#ë8Upà³E
ÑVÞNµ pUMº;º¼{¼xh9ë§
+º*ÈÕ«R©(ô:[-7±>¶rTg
gË¨¼8{-PUlkåÞ)¸$7
¿F¼b×kfÔ{äY9zð_ÈGz#ËBU
Óðj§ÎëÂu`»(i ¨ìk;ãan
aäÖP¿Ïß?Rè¢[XÒCkâÐã
«ÙT3M!«âVäÃ¿
Tõq&RÏîU÷«G{gß¡Ä/
¬´p7V¥iB}¾(s»Ax=¾B±­
u1/s¾W0b]b/	eùÓÝÙ
xÏBäË}§¡zÑ¹­Ú±·YD@
¼nç7ÏTÕÕkÖ¸7Íà¤Ä^
¾ï\Õ:¢e»»D.:o«Øi¤mX
®¾¦$Õ`;=føXÑv:ú·v
ßöÕ9²Èøø"6ÈçGYEM^Á¿M·¢
âÉæn=GGLq¤Ç ÅÉGÎ	
/^ªõÆ8îîaN1 ;Þ^î¹,Ö
kõøIÔÔZ´lìøÙoa÷¦
Ã¬DÔW((S"qÕÑ2Ìt[Ä
ÍÃÒgÔ¿x8¼ú`ÄY¬{ð[ ¨Fð
 Ô¸M1~!``§~î¤3.\c£.¨W
á»XÃÃÞhF?)oÆ`isíå·%v
ÛNFøfÔ©ÄA]ÆÀºG§&*UðF
½G1$ªø:i¯C	&$hd
nä-§âäù?¡ëX
(å	»ÅK¬5PoÈXðêl¡ò
Y1¦	¯

ØßQFDï×jµEÆª4Vö×â "
5SbáÅ+¹®®òÓh³Ñõ>@
ùð."j#§MYí[ÈÑ×Sjl
n°Öz)Çcñ©Ø¬¦OóC[ÌÎ
¯vÑ×>nÆtÃÒ¶-È|¿1Ý2íIí
Mðd¿Ú¬Z*·ÂÖCe 8½
¥Ö®zÊU ©ãöm_ñ ÂK7Û
n#°A¶7®éz2Jå\?ÄæÚ_Üc
qñÁ'ñW8áéÉZí0RââbÅn
ÆúmÄËø¼´D~òF»zd^çG¨5..
gÆ²¹ Å*dË¡]t¾< {M£áùù
_&oMy³ÄÍ F9G}Øäô
(n×µôg¡©¡4ê5¼rÙ¡¥bHH
h­åªªt±ë³_ô
³mÆÒÄ¬ÇÔ9ô Rúlsº`xkÐ
ïÕ$À~ÇØºÍsprVfÍ¬CXmV
LÈÈg=®nÆÜ[?+Vìñ§=kt
Üß[£¥÷\«{J.®:2h· 
¯ðA d°F¦×tXèp
g¶]z´ôß6:3wO
^ÏÈöjn`çL®Ö;0ãÜÉnÓÍG
¶qEP
GD¯;µÚ¿$ðl5+Ëoyàe=
ÖÓÞF¯)kTA®x~Oï0>vi
©¹ÄnFçsÚ°ÀØ=Ã1¶[8Á÷óË
¢²p|ÞuÃÚé·%5 4zU
Þ^9êÄ[2¶ETh9è_V%
;Æ´¨¦¨cªâ.÷g±ÅÐârtC
Ë;â¼z:IæÔ3°Õé o
áÜ!÷?ÏePâéÈG$µcAU/Ñ
'Ã¼¹ä³eª¿áFâXC Cjð
S!ª§·K§=è@A|wÀ±°OÀ
«NÅôñ?uîè|Ó&ÑÅÐéÔ\«ÃmÞ
tÜ{ÝLÖa:ÀðøpY
wiímjÓð%ÒÐ{¹?êÓwÌyd%óÚ q
Ìr³½gR1oÙ3sæjóÊ=5YæFd1
Úòø¡À:cU²BÁí)i´^4
eGë`ó9f¤«ï½+ª²­oÆê'®qR
.á²Õ~`T$:Ø¸êãs~K
1´Ã»dô;Üê£DÃ_kVzëöà­
BFcåÙ©

õo[5ïñà<Î6ËF>ó»

°öòs	²nt¡¾ømXøL¡4
RÔ8¼
F^äàv×ÃTZzÄéRW*ÕhªÀ
×3/£©sG­ÒÛ~7ÅFº2&ª
g$Á½RòX©ÁÇ[éÃRÖ
jò/§£ª¼ÛáÇz-ÆëåzÂUI´=
¿ ÖIìøç~Ð2ârk
àæ~JÐUø
 h47ÚZÍd©ÉÂ?Ä²~uÃÈèÈ
X'Ø)7ß!³¢î1²«ÄÃÛ½Ûæ
!oï·s=¡¡Lîf8Íq7
$BõË?áYëöéËÄukÕ
òy
¬n4W±`w|9Ò±Íd÷ïGò{ðôU
èÖEcJ!b«_Ï)»ñN%+¯8c?\%
E6éò±n¤È ¤¶ÒyÏ]RzC
ÝM"|?pP\gb>Â¬5"ã¦o
Ñ°b/ôé¦øÃá+½u¥i
hØ9¡"àï6IÁ
,=[1nf
WÐ6{'Tçk8 »di`q ´6é
¯ÚÅq¡W;d¡>vè½½ÎÇT
x×ñXÝzáE)I@ eáòX^]8
{ª>å:9s¸$¢S¹M^`áÞâ
¿ÌÅ¬ù/h5·£­|åp1M³»	W
 9#ç2iy|AÇ+çï©Å×[P
XóÇä6Í'OjìunÆ*°MIQE¿êyn
-Ð?Èmu¥
$ï]ªfNQ¢)Éi¬CÉ Á{ÔÜ
4TÖølÃcó
§mÝJBî¶ëÊ â 
p¼4æZÆ6	ëcå9¡<M$5-Të
È{Ø5¤fïÚ?Ü\S·"Ñè`Ì@r
ÃïÃ7¢\8-ÊAíy6¥MÑ:
©úÜw°N;£ÏäëÐW
é¬Kfäá>±¹Wâ#ïÖUÈà[#i\
*
9Tç´OiYu>zé53 Ö9,
Æ®r%¸æÂ¡'c®¹^véWJ
KÅ1ÃÚ~¦ÔpR6äÐXPv
NáÖ÷ 'ð#ß½@|ké`ÓåØ
Ö
Y+°ÑQl»Û&Báå±A¾Ø´
zcG³¤bùR3DsÛDuJ¾1M¨
o9cSb8M¡p\ ÈãÒ§Z+`Æ¢
8#$ñÊ;r;

% ï"îR&Éúç××ãdhËø¼õñ
éh°'èÕ¼>>
ñk±ïÃ¹¼"
ì;gæMË7$0½4 ¶d[/Á	#
öî >RxkÅ·Ë?§îzoË|Ì
7[ï,UrQeéèñX£)\L
Xvð0_ ´xç­z¬aÀøojº
Xb¯æ	1¸¢||nÖ5¥õ)½æ
[5ÀÌêWaØìës ,¨ìEøM
Ó·2YÂ¡ãL%æÀtÐ7w+¡Ë;È¼R
$G ²É2ù>3ËÊU_Õp©"
lÞ4Ê»Sò§_´©¼7£&ª@
5+K]$÷»áo×FÁï£Eòà[Èh·Öl
8ù\C
TzU¹K¢EÍbÇ¥LÂÐë==Î
H?CÞÇ±1¸<Ë¸Nôàc°Wµí
§Ìµ´bB"à#$¥~S¾ä÷rd½
ÜfET íqÙPøC×ôFlw£Û
Ï\©qÛ_¾+²ç}Æ`jT1
¨mxèG*©26§Ñ\´6n
1ÁÒw6F;?ë½'6Ü@Æ"Ç.
>òr¯Õà,~0;]túÂujù
 :gð¿±Ñ\^o%
_ù2÷±·YiíìVxêúõ4¶´H
bÌCÝrÅ`£³aö Ãq­aç'·<ª
êør?Æ=º (ëÛ´÷à¢>
+eË-t¡Ð¬âlØtw"J¹è0ÙV
$tw¾Aq 6Õ	]`âÎJ!÷t
Ll
Q}öÊé$ïtkLOjù[n±ÝÏ-( 
O?ë7Õµs8ncÐêvyµÕåW®
YòuA lÉøíøí$é¹¨Þ³ãO
_tc÷DÚ1R¯ÑEØ\ç`"íMQù³
í­Fß¡Ê®GÁE¬äVOäKÑ=
¶f4@Ô `ì¯M©°44£s¥¢i
¹9E&ºx¿	Ù;#©-Ú^RÚ­(dË
eúP"OudM IÏ¥ÁÞRÊ
¡g©	"J&ó± + ¼³?ÀNæ
ù&RZSµÛ\ÒaÝd¤ãÖºa¸	
ÂnièáñHµÀ DxçÏ^9òä5
ÅAzÎÇN^Ñs\Ã¢-7V sxK
Õan«rT4A©YL·a&YÃ¶
Ø¢